In [302]:
import json
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime
%matplotlib inline
%pprint

Pretty printing has been turned ON


In [303]:
path = 'data/kaggle/'
data = pd.read_csv(path + 'train.csv',
                   parse_dates=['Date'],
                   dtype={
                       'Id': 'int',
                       'Province_State': 'str',
                       'Country_Region': 'str',
                       'ConfirmedCases': 'float',
                       'Fatalities': 'float'
                   })
data.replace([np.inf, -np.inf], np.nan)
print(data.shape)
data

(22644, 6)


,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,2020-01-26,0.0,0.0
...,...,...,...,...,...,...
22639,32705,NaN,Zimbabwe,2020-03-31,8.0,1.0
22640,32706,NaN,Zimbabwe,2020-04-01,8.0,1.0
22641,32707,NaN,Zimbabwe,2020-04-02,9.0,1.0
22642,32708,NaN,Zimbabwe,2020-04-03,9.0,1.0


In [304]:
print(data['Province_State'].describe())
xs = data.groupby(['Province_State', 'Date'])['ConfirmedCases'].sum()
xs = xs.iloc[xs.index.get_level_values('Date') == datetime.datetime(2020,4, 4)]
ix = np.argsort(-1*xs)
print()
print('=================================================')
print('****** TOP 30 States with confirmed cases as of AM ******')
xs[ix][:30]

count           9620
unique           130
top       Montserrat
freq              74
Name: Province_State, dtype: object

****** TOP 30 States with confirmed cases as of AM ******


Province_State   Date      
New York         2020-04-04    113833.0
Hubei            2020-04-04     67803.0
New Jersey       2020-04-04     34124.0
Michigan         2020-04-04     14225.0
California       2020-04-04     12837.0
Louisiana        2020-04-04     12496.0
Massachusetts    2020-04-04     11736.0
Florida          2020-04-04     11545.0
Pennsylvania     2020-04-04     10444.0
Illinois         2020-04-04     10357.0
Washington       2020-04-04      7247.0
Texas            2020-04-04      6556.0
Georgia          2020-04-04      6160.0
Quebec           2020-04-04      6101.0
Connecticut      2020-04-04      5276.0
Colorado         2020-04-04      4188.0
Indiana          2020-04-04      3953.0
Ohio             2020-04-04      3739.0
Ontario          2020-04-04      3630.0
Tennessee        2020-04-04      3322.0
Maryland         2020-04-04      3125.0
New South Wales  2020-04-04      2493.0
North Carolina   2020-04-04      2486.0
Virginia         2020-04-04      2407.0
Missouri    

more than half province state is NA, dropping province state sounds ok, we can link population density later if needed.

In [305]:
#data.drop('Province_State', axis=1, inplace=True)
data.drop('Id', axis=1, inplace=True)

### EDA

In [306]:
cases = data.groupby(['Country_Region', 'Date'])['ConfirmedCases'].sum()
cases_latest = cases.iloc[cases.index.get_level_values('Date') == datetime.datetime(2020,4, 4)]
ind = np.argsort(cases_latest)[::-1]
cases_latest = cases_latest[ind]
deaths = data.groupby(['Country_Region', 'Date'])['Fatalities'].sum()
deaths_latest = deaths.iloc[deaths.index.get_level_values('Date') == datetime.datetime(2020,4, 4)]

In [307]:
fatality_rate_bycountry = deaths_latest / cases_latest

In [308]:
indf = np.argsort(fatality_rate_bycountry)[::-1]
fatality_rate_bycountry = fatality_rate_bycountry[indf]

In [309]:
top_35_fatal = fatality_rate_bycountry[:35]
top_20_cases = cases_latest[:20]

In [310]:
top_35_fatal

Country_Region       Date      
Botswana             2020-04-04    0.250000
Gambia               2020-04-04    0.250000
MS Zaandam           2020-04-04    0.222222
Sudan                2020-04-04    0.200000
Angola               2020-04-04    0.200000
Nicaragua            2020-04-04    0.200000
Guyana               2020-04-04    0.173913
Mauritania           2020-04-04    0.166667
Bahamas              2020-04-04    0.142857
Cabo Verde           2020-04-04    0.142857
Syria                2020-04-04    0.125000
San Marino           2020-04-04    0.123552
Italy                2020-04-04    0.123259
Congo (Kinshasa)     2020-04-04    0.116883
Bangladesh           2020-04-04    0.114286
Zimbabwe             2020-04-04    0.111111
Algeria              2020-04-04    0.103917
United Kingdom       2020-04-04    0.101704
Liberia              2020-04-04    0.100000
Suriname             2020-04-04    0.100000
Netherlands          2020-04-04    0.099013
Spain                2020-04-04    0.094691


In [311]:
top_20_cases

Country_Region  Date      
US              2020-04-04    308690.0
Spain           2020-04-04    126168.0
Italy           2020-04-04    124632.0
Germany         2020-04-04     96092.0
France          2020-04-04     90848.0
China           2020-04-04     82543.0
Iran            2020-04-04     55743.0
United Kingdom  2020-04-04     42476.0
Turkey          2020-04-04     23934.0
Switzerland     2020-04-04     20505.0
Belgium         2020-04-04     18431.0
Netherlands     2020-04-04     16725.0
Canada          2020-04-04     12965.0
Austria         2020-04-04     11781.0
Portugal        2020-04-04     10524.0
Brazil          2020-04-04     10360.0
Korea, South    2020-04-04     10156.0
Israel          2020-04-04      7851.0
Sweden          2020-04-04      6443.0
Norway          2020-04-04      5550.0
Name: ConfirmedCases, dtype: float64

Aside from Spain and Italy, top 20 fatalaties do not have the highest number of cases, in fact it seems African/Developed coutries, related to health care system? most likely, 
we can check this dataset later: https://www.kaggle.com/danevans/world-bank-wdi-212-health-systems

other factor could be pollution, China has considerably higher pollution rate than most countries but health care and tech advance plays a huge role clearly. We'll see...

## Data Featuring

In [312]:
data['weekday'] = data['Date'].dt.weekday
data['day'] = data['Date'].apply(lambda x: int(x.strftime('%j')))

weekday_name = data['Date'].dt.weekday_name  # for reference if needed

In [313]:
data

,Province_State,Country_Region,Date,ConfirmedCases,Fatalities,weekday,day
0,NaN,Afghanistan,2020-01-22,0.0,0.0,2,22
1,NaN,Afghanistan,2020-01-23,0.0,0.0,3,23
2,NaN,Afghanistan,2020-01-24,0.0,0.0,4,24
3,NaN,Afghanistan,2020-01-25,0.0,0.0,5,25
4,NaN,Afghanistan,2020-01-26,0.0,0.0,6,26
...,...,...,...,...,...,...,...
22639,NaN,Zimbabwe,2020-03-31,8.0,1.0,1,91
22640,NaN,Zimbabwe,2020-04-01,8.0,1.0,2,92
22641,NaN,Zimbabwe,2020-04-02,9.0,1.0,3,93
22642,NaN,Zimbabwe,2020-04-03,9.0,1.0,4,94


In [314]:
cases, fatalities = data['ConfirmedCases'], data['Fatalities']  # Get y_1, y_2
data.drop('ConfirmedCases', axis=1, inplace=True)
data.drop('Fatalities', axis=1, inplace=True)

### Join density data
source = https://worldpopulationreview.com/countries/countries-by-density/#dataTable
* `rank`: country rank
* `name`: country name
* `density`: density /km^2
* `densityMi`: density /mi^2
* `pop2020`: Populatio as of 2020
* `Area`: Area in km^2

In [315]:
dens = {}
with open('data/rho.json', 'r') as js:
    d = json.load(js)['data']
ds = pd.DataFrame(d)
ds.head()

,rank,name,density,densityMi,pop2020,area
0,149,Afghanistan,59.6850,154.5841,38928.3460,652230
1,110,Albania,100.1043,259.2700,2877.7970,28748
2,198,Algeria,18.4113,47.6854,43851.0440,2381741
3,50,American Samoa,277.3417,718.3150,55.1910,199
4,74,Andorra,165.0962,427.5990,77.2650,468


In [316]:
for js in d:
    dens[js['name']] = [js['density'], js['pop2020']]  ## /km2
data['Density'] = data['Country_Region'].apply(lambda c: float(dens[c][0]) if c in dens else float('nan'))
data['Population'] = data['Country_Region'].apply(lambda c: float(dens[c][1]) if c in dens else float('nan'))

In [317]:
data.head()

,Province_State,Country_Region,Date,weekday,day,Density,Population
0,NaN,Afghanistan,2020-01-22,2,22,59.685,38928.346
1,NaN,Afghanistan,2020-01-23,3,23,59.685,38928.346
2,NaN,Afghanistan,2020-01-24,4,24,59.685,38928.346
3,NaN,Afghanistan,2020-01-25,5,25,59.685,38928.346
4,NaN,Afghanistan,2020-01-26,6,26,59.685,38928.346


### Join health care data

In [318]:
health_care = pd.read_csv(path+'../2.12_Health_systems.csv')

In [319]:
health_care.head()

,Country_Region,Province_State,World_Bank_Name,Health_exp_pct_GDP_2016,Health_exp_public_pct_2016,Health_exp_out_of_pocket_pct_2016,Health_exp_per_capita_USD_2016,per_capita_exp_PPP_2016,External_health_exp_pct_2016,Physicians_per_1000_2009-18,Nurse_midwife_per_1000_2009-18,Specialist_surgical_per_1000_2008-18,Completeness_of_birth_reg_2009-18,Completeness_of_death_reg_2008-16
0,Afghanistan,NaN,Afghanistan,10.2,5.1,77.4,57.2,162.8,17.5,0.3,0.3,0.0,42.3,NaN
1,Albania,NaN,Albania,6.7,41.4,58.0,271.5,759.7,0.7,1.2,3.6,11.6,98.4,53.0
2,Algeria,NaN,Algeria,6.6,67.7,30.9,260.4,998.2,0.0,1.8,2.2,12.1,100.0,NaN
3,Andorra,NaN,Andorra,10.4,49.1,41.7,3834.7,4978.7,NaN,3.3,4.0,83.1,100.0,80.0
4,Angola,NaN,Angola,2.9,44.1,35.2,95.2,185.8,3.6,0.2,1.3,NaN,25.0,NaN


In [321]:
missing_countries = set(data['Country_Region']) - set(health_care['Country_Region'])
print ("Missing countries are: {}".format(missing_countries))

Missing countries are: {'Kosovo', 'Sierra Leone', 'Holy See', 'West Bank and Gaza', 'Taiwan*', 'Diamond Princess', 'MS Zaandam', 'Burma'}


In [322]:
features_to_extract = list(health_care.columns)[3:]
countries = set(health_care['Country_Region'])
health_js = {}
for i, rec in health_care.iterrows():
    if rec['Country_Region'] in countries:
        health_js[rec['Country_Region']] = [
            rec[feat] for feat in features_to_extract
        ]

for i, feat in enumerate(features_to_extract):
    data[feat] = data['Country_Region'].apply(
        lambda c: health_js[c][i] if c in health_js else float('nan'))
    

In [323]:
data

,Province_State,Country_Region,Date,weekday,day,Density,Population,Health_exp_pct_GDP_2016,Health_exp_public_pct_2016,Health_exp_out_of_pocket_pct_2016,Health_exp_per_capita_USD_2016,per_capita_exp_PPP_2016,External_health_exp_pct_2016,Physicians_per_1000_2009-18,Nurse_midwife_per_1000_2009-18,Specialist_surgical_per_1000_2008-18,Completeness_of_birth_reg_2009-18,Completeness_of_death_reg_2008-16
0,NaN,Afghanistan,2020-01-22,2,22,59.6850,38928.346,10.2,5.1,77.4,57.2,162.8,17.5,0.3,0.3,0.0,42.3,NaN
1,NaN,Afghanistan,2020-01-23,3,23,59.6850,38928.346,10.2,5.1,77.4,57.2,162.8,17.5,0.3,0.3,0.0,42.3,NaN
2,NaN,Afghanistan,2020-01-24,4,24,59.6850,38928.346,10.2,5.1,77.4,57.2,162.8,17.5,0.3,0.3,0.0,42.3,NaN
3,NaN,Afghanistan,2020-01-25,5,25,59.6850,38928.346,10.2,5.1,77.4,57.2,162.8,17.5,0.3,0.3,0.0,42.3,NaN
4,NaN,Afghanistan,2020-01-26,6,26,59.6850,38928.346,10.2,5.1,77.4,57.2,162.8,17.5,0.3,0.3,0.0,42.3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22639,NaN,Zimbabwe,2020-03-31,1,91,38.0362,14862.924,9.4,46.5,21.2,93.9,185.0,25.4,0.1,1.2,1.6,38.0,NaN
22640,NaN,Zimbabwe,2020-04-01,2,92,38.0362,14862.924,9.4,46.5,21.2,93.9,185.0,25.4,0.1,1.2,1.6,38.0,NaN
22641,NaN,Zimbabwe,2020-04-02,3,93,38.0362,14862.924,9.4,46.5,21.2,93.9,185.0,25.4,0.1,1.2,1.6,38.0,NaN
22642,NaN,Zimbabwe,2020-04-03,4,94,38.0362,14862.924,9.4,46.5,21.2,93.9,185.0,25.4,0.1,1.2,1.6,38.0,NaN


we just need to fill Density, Health GDP, and physicians with mean from other countries and create new columns

In [324]:
for column in data:
    print(column)
    print(
        f"\t\t dtype: {data[column].dtype}, # NA's{data[column].isnull().sum()}"
    )
    print('----------------------------------------------------------------')

Province_State
		 dtype: object, # NA's13024
----------------------------------------------------------------
Country_Region
		 dtype: object, # NA's0
----------------------------------------------------------------
Date
		 dtype: datetime64[ns], # NA's0
----------------------------------------------------------------
weekday
		 dtype: int64, # NA's0
----------------------------------------------------------------
day
		 dtype: int64, # NA's0
----------------------------------------------------------------
Density
		 dtype: float64, # NA's5106
----------------------------------------------------------------
Population
		 dtype: float64, # NA's5106
----------------------------------------------------------------
Health_exp_pct_GDP_2016
		 dtype: float64, # NA's8584
----------------------------------------------------------------
Health_exp_public_pct_2016
		 dtype: float64, # NA's8584
----------------------------------------------------------------
Health_exp_out_of_pocket_pct_2016
		 d

In [325]:
means = {} 

data['Province_NA'] = data['Province_State'].isnull()

for column in ['Density', 'Population'] + features_to_extract:
    data[column+'_NA'] = data[column].isnull()
    m = data[column].mean()
    data[column] = data[column].fillna(m)
    means[column] = m

In [326]:
data['ConfirmedCases'], data['Fatalities'] = cases, fatalities

In [327]:
data.to_feather('data/covid19_data')